In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import numpy as np

credentials = service_account.Credentials.from_service_account_file(
'/Users/joaolousada/Downloads/solar-vertex-380321-55f7948b7f6a.json') 

project_id = 'solar-vertex-380321'
client = bigquery.Client(credentials= credentials,project=project_id)

In [4]:
import pandas_gbq

sql = """
SELECT *
FROM solar-vertex-380321.zendesk.zentest
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [5]:
df.head()

,EmployeeID,TicketChannel,TicketGroup,TicketID,TicketCreatedDate,ZendeskPlan,DayTicketSolved,AssigneeTier,FirstReplyTime,FullResolutionTime,NumAgentComments,NumPublicComments,TicketEscalated,CustomerSatisfaction,WorkerDateHired,WorkerLocation,Worker_Type,WorkerTenure
0,938,chat,Support (T1),2536525,2017-04-20,Enterprise Legacy,None,Tier 1,317,<NA>,11,15,0,<NA>,2014-11-10,Dublin,Employee,892
1,938,chat,Support (T1),2522091,2017-04-14,Enterprise Legacy,None,Tier 1,10260,<NA>,14,22,0,<NA>,2014-11-10,Dublin,Employee,886
2,1247,web,Support (T1),2836287,2017-08-03,Enterprise Legacy,None,Tier 1,<NA>,<NA>,4,<NA>,0,<NA>,2015-04-20,Dublin,Employee,836
3,919,help_center,Support (T1),3003180,2017-10-02,Enterprise Legacy,None,Tier 1,<NA>,<NA>,3,1,0,<NA>,2014-10-27,Madison,Employee,1071
4,919,help_center,Support (T1),3003011,2017-10-02,Enterprise Legacy,None,Tier 1,<NA>,<NA>,3,1,0,<NA>,2014-10-27,Madison,Employee,1071


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240665 entries, 0 to 240664
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   EmployeeID            240665 non-null  Int64 
 1   TicketChannel         240665 non-null  object
 2   TicketGroup           240665 non-null  object
 3   TicketID              240665 non-null  Int64 
 4   TicketCreatedDate     240665 non-null  dbdate
 5   ZendeskPlan           177729 non-null  object
 6   DayTicketSolved       239741 non-null  object
 7   AssigneeTier          208010 non-null  object
 8   FirstReplyTime        157169 non-null  Int64 
 9   FullResolutionTime    239741 non-null  Int64 
 10  NumAgentComments      195237 non-null  Int64 
 11  NumPublicComments     192623 non-null  Int64 
 12  TicketEscalated       240665 non-null  Int64 
 13  CustomerSatisfaction  45554 non-null   Int64 
 14  WorkerDateHired       240665 non-null  dbdate
 15  WorkerLocation   

In [23]:
q1 = f'how many tickets were created ?\n'

a1 = df['TicketID'].nunique()
print(q1,a1)

q2 = f'--- \nhow % of tickets were solved ?\n'

a2 = df['FullResolutionTime'].count()
print(q2,np.round(100*a2/a1, 2))

q3 = f'--- \nhow long does it take to solve a ticket ?\n'
print(q3, df['FullResolutionTime'].describe())



how many tickets were created ?
 240665
--- 
how % of tickets were solved ?
 99.62
--- 
how long does it take to solve a ticket ?
 count    239741.000000
mean       5949.678053
std       19227.491206
min           0.000000
25%          13.000000
50%         116.000000
75%        8112.000000
max      548814.000000
Name: FullResolutionTime, dtype: float64
